In [1]:
import os

In [2]:
%pwd

'd:\\pythonProjects\\MLflow_ML_project\\research'

In [3]:
os.chdir('../')
%pwd

'd:\\pythonProjects\\MLflow_ML_project'

In [4]:
# entity 
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path 
    source_URL: str
    local_data_file: Path
    unzip_dir: Path 

In [5]:
# configuration 
from mlProject.constants import * 
from mlProject.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(
         self,
         config_filepath=CONFIG_FILE_PATH,
         params_filepath=PARAMS_FILE_PATH,
         schema_filepath=SCHEMA_FILE_PATH   
    ):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )
        return data_ingestion_config

In [6]:
# components
import os 
import urllib.request as requests 
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size
import gdown 

class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config 

    def download_file(self):
        try:
            dataset_url=self.config.source_URL
            zip_download_dir=self.config.local_data_file
            os.makedirs('artifacts/data_ingestion',exist_ok=True)
            logger.info(f'Downloading data from {dataset_url} into file {zip_download_dir}')

            file_id=dataset_url.split('/')[-2]
            prefix='https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)
            logger.info(f'Downloaded data from {dataset_url} into file {zip_download_dir}')

        except Exception as e:
            raise e

    def extract_unzip_file(self):
        """ 
        extract the downloaded zip
        returns: None
        """
        unzip_path=self.config.unzip_dir
        os.makedirs(self.config.unzip_dir,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [10]:
# pipeline 
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_unzip_file()
except Exception as e:
    raise e 

[2025-01-03 07:15:16,887: INFO:common: yaml file: config\config.yaml loaded successfully]
[2025-01-03 07:15:16,889: INFO:common: yaml file: params.yaml loaded successfully]
[2025-01-03 07:15:16,890: INFO:common: yaml file: schema.yaml loaded successfully]
[2025-01-03 07:15:16,891: INFO:common: created directory at: artifacts]
[2025-01-03 07:15:16,892: INFO:common: created directory at: artifacts/data_ingestion]
[2025-01-03 07:15:16,893: INFO:2800078568: Downloading data from https://drive.google.com/file/d/1cEAi27z0Fd0GlbZ6esrsxXJbe95glDaI/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From: https://drive.google.com/uc?/export=download&id=1cEAi27z0Fd0GlbZ6esrsxXJbe95glDaI
To: d:\pythonProjects\MLflow_ML_project\artifacts\data_ingestion\data.zip
100%|██████████| 21.5k/21.5k [00:00<00:00, 21.5MB/s]

[2025-01-03 07:15:20,925: INFO:2800078568: Downloaded data from https://drive.google.com/file/d/1cEAi27z0Fd0GlbZ6esrsxXJbe95glDaI/view?usp=sharing into file artifacts/data_ingestion/data.zip]


In [8]:
from mlProject.constants import * 
CONFIG_FILE_PATH

WindowsPath('config/config.yaml')

In [9]:
%pwd

'd:\\pythonProjects\\MLflow_ML_project'